In [ ]:
from nbfix import load_catalog

from typing import Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
catalog = load_catalog()

X = catalog.load("train.features")
y = catalog.load("train.pressure")

df = pd.concat([X, y], axis=1)
display(df)

num_cycles = df["cycle"].max()

In [ ]:
from scipy.integrate import cumtrapz

def compute_velocity(df: pd.DataFrame) -> pd.DataFrame:
    def _compute_velocity_per_cycle(grp: pd.DataFrame):
        vel = cumtrapz(grp["acc"].abs(), grp["phi"])
        vel = np.insert(vel, 0, 0)

        grp["vel"] = vel
        return grp

    # compute velocity by integrating acceleration per cycle
    df = df.groupby("cycle").apply(_compute_velocity_per_cycle)

    return df

# df = compute_velocity(df)


In [ ]:
def plot_cycle(n: int, figsize: Tuple[int, int] = (20, 5)):
    c = df[df["cycle"] == n]

    # get a 3-axes figure
    fig, ax1 = plt.subplots(figsize=figsize)
    fig.subplots_adjust(right=0.75)
    ax2, ax3 = ax1.twinx(), ax1.twinx()
    ax3.spines.right.set_position(("axes", 1.2))

    # plot the data
    ax1.plot(c.phi, c.pa, c="green")
    ax1.set_ylabel("pressure")

    ax2.plot(c.phi, c.acc, c="gray", ls=":", lw=0.5, marker=".")
    ax2.set_ylabel("acceleration")

    ax3.plot(c.phi, c.vel_abs, c="orange")
    ax3.set_ylabel("velocity")

    # place dots at maximum
    pa_max_xy = c.phi.iloc[c.pa.argmax()], c.pa.max()
    ax1.plot(*pa_max_xy, c="green", marker="v", markersize=10)

    vel_max_xy = c.phi.iloc[c.vel.argmax()], c.vel.max()
    ax3.plot(*vel_max_xy, c="red", marker="^", markersize=10)

    # grids, labels, titles
    ax1.grid()
    ax1.set_xlabel("crank angle")
    ax1.set_title(f"cycle {n}")

    plt.show()

In [ ]:
for i in np.random.randint(1, num_cycles + 1, size=10):
    plot_cycle(i)
